In [1]:
# default_exp changePassword

## Change Password

> Change password of this account

In [13]:
#hide
import src.setEnv 
import requests

In [3]:
#export 
from src.userTable import UserTable
from awsSchema.apigateway import Response, Event
from copy import deepcopy
from src.saltHashPassword import hash_password, check_password
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from beartype import beartype

## Change Password Input Class

In [4]:
#export 
@dataclass_json
@dataclass
class ChangePasswordInput:
    #change to username
    # userId: str
    username: str
    oldPassword: str
    newPassword: str
    newPasswordRepeated: str
    
    @property
    def passwordHash(self):
        return hash_password(self.newPassword)
    
    def checkPassword(self):
        for item in UserTable.username_index.query(self.username):
            return check_password(item.passwordHash, self.oldPassword)
        

## Main Lambda Handler

In [5]:
#export
class H:
    class ParseInputError(Exception): pass
    class PasswordCheckError(Exception): pass
    class InvalidPasswordError(Exception): pass
    class IncorrectRepeatedPasswordError(Exception): pass
    class HashPasswordError(Exception): pass
    class ChangePasswordError(Exception): pass
    
    @classmethod
    @beartype
    def parseInput(cls, event: dict)->ChangePasswordInput:
        try:
            user = Event.parseDataClass(ChangePasswordInput, deepcopy(event))
            return user
        except Exception as e:
            raise cls.ParseInputError(e)
    
    @classmethod
    @beartype
    def changeToNewPassword(cls, user: ChangePasswordInput):
        try:
            passwordChecked = user.checkPassword()
        except Exception as e:
            raise cls.PasswordCheckError(e)
        if passwordChecked:
            pass
        else:
            raise cls.InvalidPasswordError
        if user.newPassword == user.newPasswordRepeated:
            pass
        else:
            raise cls.IncorrectRepeatedPasswordError()
        try:
            newPasswordHashed = hash_password(user.newPassword)
        except Exception as e:
            raise cls.HashPasswordError(e)
        try:
            usernameQuery = UserTable.username_index.query(user.username)
            for i in usernameQuery:
                userId = i.userId
                oldItem = UserTable.get(userId)
                oldItem.passwordHash = newPasswordHashed
                oldItem.save()        
        except Exception as e:
            raise cls.ChangePasswordError(e)
        return True

In [6]:
#export 
def changePassword(event, *args):
    try:
        user = H.parseInput(event)
        H.changeToNewPassword(user)
        return Response.returnSuccess()
    except H.ParseInputError as e:
        return Response.returnError(f'failed to parse input {e}')
    except H.PasswordCheckError as e:
        return Response.returnError(f'failed to check password {e}')
    except H.InvalidPasswordError as e:
        return Response.returnError(f'incorrect old password inputted {e}')
    except H.IncorrectRepeatedPasswordError as e:
        return Response.returnError(f'new password does not match repeated new password {e}')
    except H.HashPasswordError as e:
        return Response.returnError(f'failed to hash password {e}')
    except H.ChangePasswordError as e:
        return Response.returnError(f'unable to change password successfully {e}')
    except Exception as e:
        return Response.returnError(f' unknown error {e}')

In [20]:
input_ = {'username': 'Delg45678', 'oldPassword': '12765', 'newPassword': '12764', 'newPasswordRepeated':'12764'}
event = Event.getInput(body = input_)
changePassword(event)

{'body': '{"error":"incorrect old password inputted "}',
 'statusCode': 400,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

In [19]:
url = 'https://3d4v07deqh.execute-api.ap-southeast-1.amazonaws.com/Prod/changepassword'
body = {'username': 'Delg45678', 'oldPassword': '12765', 'newPassword': '12764', 'newPasswordRepeated': '12764'}
r = requests.post(url, json=body)
print(r.text)

{"error":"incorrect old password inputted "}
